## Importar Librerias

In [7]:
import pandas as pd
from pymongo import MongoClient
import re
from datetime import datetime
import json

## Extraemos los Datos

In [ ]:
def extraer_datos(path, mongo_uri="mongodb://localhost:27017/", db_name="mi_basedatos", collection_name="mi_coleccion"):
    try:
        data = pd.read_json(path)
        return data
    except FileNotFoundError:
        print("Fichero no encontrado localmente, buscando en tu base de datos...")
        try:
            client = MongoClient(mongo_uri)
            db = client[db_name]
            collection = db[collection_name]
            
            documentos = list(collection.find())
            if documentos:
                return pd.DataFrame(documentos)
            else:
                return "No se encontraron datos en MongoDB."
        except Exception as e:
            return f"Error al conectar con MongoDB: {e}"


In [6]:
# Parametros
MONGO_URI = "mongodb://localhost:27020/"
DB_NAME = "PIA"
COLLECTION_NAME = "peticions"
PATH = "../res/mobiles.json"

In [ ]:
data = extraer_datos(path=PATH, mongo_uri=MONGO_URI ,db_name=DB_NAME,collection_name=COLLECTION_NAME)
data.set_index('_id', inplace=True)
data

,ASIN,Actualizaciones de software garantizadas hasta,Ano del modelo,Aparatos compatibles,Capacidad de almacenamiento de memoria,Capacidad de almacenamiento digital,Capacidad de la memoria,Capacidad de la memoria RAM,Capacidad de la memoria RAM instalada,Capacidad de la memoria flash instalada,...,Generacion,GPU,Energia de la bateria,Brillo de imagen,Compatibilidad del montaje,Tasa de contraste de imagen,Tipo de fuente de luz,Tipo de montaje,Fabricante del procesador de graficos,Marca de la tarjeta grafica
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,B00IRZ8EQC,desconocido,2013.0,microSD (TransFlash),8 GB,"1,7 Modificador desconocido",8 GB,1536 MB,"1,5 GB","1,5 GB",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00J8OA220,B00J8OA220,desconocido,NaN,NaN,160 GB,NaN,160 GB,NaN,2 GB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00JC8MD7Y,B00JC8MD7Y,NaN,NaN,NaN,6 GB,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TKALUDC,B00TKALUDC,desconocido,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TUXHZTW,B00TUXHZTW,desconocido,2015.0,NaN,12 GB,128 GB,12 GB,NaN,16 GB,128 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,B0DH3N1GZ3,desconocido,NaN,NaN,128 GB,NaN,128 GB,NaN,4 GB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09QH71RZR,B09QH71RZR,NaN,NaN,NaN,128 GB,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0DK4NBZWB,B0DK4NBZWB,desconocido,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Eliminamos las columnas que no aportan valor

In [3]:
columnas_excluidas = {
    "Numero de pilas de ion de litio", 
    "Que hay en la caja?",
    "Formatos de audio compatibles",
    "Otras caracteristicas",
    "Otras caracteristicas",
    "Adaptador",
    "Formatos compatibles", 
    "Tiene estabilizador de imagen", 
    "Tipo de medio", 
    "Aparatos compatibles",
    "Descripcion de la interfaz de red",
    "Caracteristicas especiales",
    "Tecnologia del microfono", 
    "Tecnologia de comunicacion inalámbrica",
    "Grabador",
    "Formatos de audio compatibles",
    "Series",
    "Carga electrica de la bateria",
    "OS",
    "Tiempo de carga",
    "Conector de audio",
    "Resolucion de la camara web trasera",
    "Pilas incluidas",
    "Software incluido",
    "Tamano",
    "Tipo de conector",
    "Ranuras de memoria disponibles",
    "Composicion de la pila de la bateria",
    "Fuente de alimentacion",
    "Proveedor de conexion inalambrica",
    "Velocidad de transferencia de datos", 
    "Descripcion de la bateria", 
    "Duracion de la bateria media (en horas)", 
    "Marca de la tarjeta grafica",
    "Zoom digital",
    "Enfoque automatico",
    "Tiempo de carga de la bateria (en horas)",
    "Memoria extraible",
    "Proveedor de conexion inalambrica",
    "Tamano de la tarjeta de memoria incluida",
    "Relacion de aspecto de imagen", 
    "Dimensiones del paquete", 
    "Embalaje de la bateria de litio", 
    "Tipo de dispositivo", 
    "Actualizaciones de software garantizadas hasta", 
    "Tecnologia de comunicacion inalambrica",
    "Compilador", 
    "Numero de pilas de metal de litio", 
    "Voltaje", 
    "Tiempo de conversacion telefonica", 
    "Numero de puertos USB", 
    "Contenido de energia de la bateria de litio", 
    "Lector", 
    "Marimba", 
    "Tipo de material", 
    "Incluye bateria recargable", 
    "Tiene vision nocturna", 
    "Pantalla a color", 
    "Proveedor de conexion inalámbrica", 
    "Tipo de lente",
    "Descripcion de la tarjeta grafica", 
    "GPS", 
    "Otras caracteristicas de la pantalla", 
    "Tipo de conexion inalámbrica", 
    "Numero de puertos", 
    "Pilas", 
    "Numero de modelo del producto", 
    "Tecnologia de conectividad", 
    "Entrada de interfaz humana", 
    "Otras funciones de la camara", 
    "Color", 
    "Compatible con tecnologia Bluetooth", 
    "0",
    "Referencia del fabricante", 
    "Tipo de conexion inalambrica", 
    "Pilas / baterias incluida", 
    "Pilas / baterias necesarias",
    'Peso del producto',
    'Resolucion del escaner',
    'Numero de producto',
    'Componentes incluidos',
    'Numero de productos',
    'Resolucion del sensor optico',
    'Tecnologia GSM',
    'Factor de forma',
    'Peso de la bateria de litio',
    'Hora de espera del telefono (con datos)',
    'Tecnologia del sintonizador',
    'Contiene liquidos',
    'Duracion media de la pila en stand-by',
    'Transcriptor',
    "Modelo",
    'Duracion media de la bateria',
    'Pilas / baterias incluidas', 
    'Tipo de memoria del ordenador', 
    'Coprocesador grafico', 
    'Nombre del modelo', 
    'Capacidad de la memoria flash instalada',
    'Material', 
    'Sistema operativo', 
    'ASIN', 
    'Operador de servicios inalambricos', 
    'Clasificacion en los mas vendidos de Amazon', 
    'Operador de servicios inalambricos', 
    'Producto en Amazones desde',
    'Restricciones de envio', 
    'titulo',
    'Tecnologia de movil'
}

df = data.drop(columns=[col for col in columnas_excluidas if col in data.columns])
df

,Ano del modelo,Capacidad de almacenamiento de memoria,Capacidad de almacenamiento digital,Capacidad de la memoria,Capacidad de la memoria RAM,Capacidad de la memoria RAM instalada,Capacidad del disco duro,Capacidad maxima de la memoria RAM,Descripcion del teclado,Dimensiones del producto,...,Interfaz del hardware,Generacion,GPU,Energia de la bateria,Brillo de imagen,Compatibilidad del montaje,Tasa de contraste de imagen,Tipo de fuente de luz,Tipo de montaje,Fabricante del procesador de graficos
_id,,,,,,,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,8 GB,"1,7 Modificador desconocido",8 GB,1536 MB,"1,5 GB",8,64 GB,tactil,"10,41 x 5,84 x 1,27 cm; 77,11 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00J8OA220,NaN,160 GB,NaN,160 GB,NaN,2 GB,NaN,NaN,NaN,"25,4 x 22,86 x 10,16 cm; 318 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00JC8MD7Y,NaN,6 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"16,36 x 7,6 x 0,77 cm; 179 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TKALUDC,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,8 x 4,8 x 9,3 cm; 75 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00TUXHZTW,2015.0,12 GB,128 GB,12 GB,NaN,16 GB,NaN,NaN,NaN,"4,6 x 1,9 x 9,1 cm; 74 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,NaN,128 GB,NaN,128 GB,NaN,4 GB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09QH71RZR,NaN,128 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7,38 x 0,81 x 15,98 cm; 179 g",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0DK4NBZWB,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Unimos columnas similares

In [4]:
fusion_columnas = {
    "Memoria RAM": ["Tamano de memoria RAM instalada","Capacidad de la memoria RAM instalada", "RAM","Memoria RAM", "Capacidad de la memoria RAM"],
    "Marca": ["Marca", "Fabricante"],
    "Tipo de pantalla": ["Tipo de pantalla", "Tecnologia de pantalla"],
    "Capacidad de almacenamiento digital": ["Capacidad de la memoria", "Capacidad de almacenamiento digital", "Capacidad de almacenamiento de memoria"],
    "Tamano de la pantalla": ["Tamano de pantalla", "Tamano de la pantalla"],
    "Resolucion": ["Resolucion","Maxima resolucion",'Resolucion maxima']
}
df_col_unidas = df.copy()
for col_final, columnas_equivalentes in fusion_columnas.items():
    columnas_presentes = [col for col in columnas_equivalentes if col in df.columns]

    if columnas_presentes:
        nueva_columna = []
        for index, fila in df.iterrows():
            encontrado = False
            for col in columnas_presentes:
                if pd.notna(fila[col]):
                    nueva_columna.append(fila[col])
                    encontrado = True
                    break
            if not encontrado:
                nueva_columna.append(None)

        df_col_unidas[col_final] = nueva_columna

        for col in columnas_presentes:
            if col != col_final:
                df_col_unidas.drop(columns=col, inplace=True)

In [5]:
df_col_unidas.columns

Index(['Ano del modelo', 'Capacidad de almacenamiento digital',
       'Capacidad del disco duro', 'Capacidad maxima de la memoria RAM',
       'Descripcion del teclado', 'Dimensiones del producto',
       'Fabricante del procesador', 'Incluye mando',
       'Incluye software para emails', 'Marca',
       ...
       'Generacion', 'GPU', 'Energia de la bateria', 'Brillo de imagen',
       'Compatibilidad del montaje', 'Tasa de contraste de imagen',
       'Tipo de fuente de luz', 'Tipo de montaje',
       'Fabricante del procesador de graficos', 'Memoria RAM'],
      dtype='object', length=103)

## Mapeo de columnas

In [6]:
df_map = df_col_unidas.copy()

In [7]:
def convertir_tb_a_gb(valor):
    if pd.notna(valor) and 'TB' in str(valor).upper():
        numero = float(valor.upper().replace('TB', '').strip())
        return numero * 1000
    return valor

In [8]:
for col in df_col_unidas.select_dtypes(include="object"):
    df_map[col] = df_col_unidas[col].str.lower()

df_map['precio_actual'] = df_map['precio_actual'].replace({',': '', '\\.': ''}, regex=True)
df_map['precio_actual'] = df_map['precio_actual'].astype(float)/100

df_map['precio_anterior'] = df_map['precio_anterior'].replace({',': '', '\\.': ''}, regex=True)
df_map['precio_anterior'] = df_map['precio_anterior'].astype(float)/100

df_map['Potencia nominal de la bateria'] = df_map['Potencia nominal de la bateria'].str.extract(r'(\d+)').astype(float)
df_map['Capacidad de almacenamiento digital'] = df_map['Capacidad de almacenamiento digital'].apply(convertir_tb_a_gb)
df_map['Capacidad de almacenamiento digital'] = df_map['Capacidad de almacenamiento digital'].str.extract(r'(\d+)').astype(float)
df_map['Resolucion horizontal'] = df_map['Resolucion horizontal'].str.extract(r'(\d+)').astype(float)
df_map['Resolucion vertical max'] = df_map['Resolucion vertical max'].str.extract(r'(\d+)').astype(float)
df_map['Memoria RAM'] = df_map['Memoria RAM'].str.extract(r'(\d+)').astype(float)

In [9]:
def convertir_tamaño(valor):
    if isinstance(valor, str):
        if 'centimetros' in valor:
            numero = float(valor.split(' ')[0].replace(',', '.'))
            return round(numero / 2.54, 2)
        elif 'pulgadas' in valor:
            numero = float(valor.split(' ')[0].replace(',', '.'))
            return round(numero, 2)
    elif isinstance(valor, (int, float)):
        return round(valor, 2)
    return None

df_map['Tamano de la pantalla'] = df_map['Tamano de la pantalla'].apply(convertir_tamaño)

In [ ]:
import pandas as pd

def estandarizar_pantalla(valor):
    if isinstance(valor, str):
        valor_lower = valor.lower()
        if 'super amoled' in valor_lower or 'superamoled' in valor_lower:
            return 'superamoled'
        elif 'amoled' in valor_lower:
            return 'amoled'
        elif 'oled' in valor_lower:
            return 'oled'
        elif 'xdr' in valor_lower:
            return 'xdr'
        elif 'ips' in valor_lower:
            return 'ips'
        elif 'lcd' in valor_lower:
            return 'lcd'
        elif 'led' in valor_lower:
            return 'led'
        elif 'dlp' in valor_lower:
            return 'dlp'
        elif 'fhd+' in valor_lower:
            return 'full hd+'
        elif 'hd+' in valor_lower:
            return 'hd plus'
        elif 'hd' in valor_lower:
            return 'high definition'
        elif '1920 x 1080' in valor_lower:
            return 'full hd'
    return valor

df_map['Tipo de pantalla'] = df_map['Tipo de pantalla'].apply(estandarizar_pantalla)

In [11]:
df_map['Velocidad de CPU'] = df_map['Velocidad de CPU'].str.strip()

def convertir_a_ghz(valor):
    try:
        if pd.isna(valor):
            return None
        if 'ghz' in valor:
            return float(valor.replace('ghz', '').replace(',', '.').strip())
        elif 'mhz' in valor:
            return float(valor.replace('mhz', '').replace(',', '.').strip()) / 1000
        elif 'hz' in valor:
            return float(valor.replace('hz', '').replace(',', '.').strip()) / 1_000_000_000
        elif 'e+' in valor:  # Para notación científica
            numero = float(valor)
            if numero >= 1e9:
                return numero / 1e9  # Está en Hz, pasarlo a GHz
            elif numero >= 1e6:
                return numero / 1e6  # Está en MHz, pasarlo a GHz
            elif numero >= 1e3:
                return numero / 1e3  # Está en KHz, pasarlo a GHz
            else:
                return numero / 1_000_000_000  # Está en Hz
        else:
            return None
    except:
        return None

df_map['Velocidad de CPU (GHz)'] = df_map['Velocidad de CPU'].apply(convertir_a_ghz)
df_map = df_map.drop("Velocidad de CPU", axis=1)

/tmp/ipykernel_96893/1013415527.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map['Velocidad de CPU (GHz)'] = df_map['Velocidad de CPU'].apply(convertir_a_ghz)


In [ ]:
def limpiar_valoracion(x):
    if isinstance(x, str) and "estrellas" in x:
        return x.split("estrellas", 1)[-1].strip()
    return x

def extraer_valoraciones_y_estrellas(texto):
    if pd.isnull(texto):
        return None, None
    
    valoraciones = re.search(r'(\d{1,3}(?:[\.,]?\d{3})*) valoraciones', texto)
    estrellas = re.search(r'(\d[\.,]?\d*) de 5 estrellas', texto)
    
    if valoraciones and estrellas:
        num_valoraciones = valoraciones.group(1).replace(',', '').replace('.', '')
        puntuacion_estrellas = estrellas.group(1).replace(',', '.')
        return int(num_valoraciones), float(puntuacion_estrellas)
    else:
        return None, None
    
df_map["valoraciones"] = df_map["Valoracion media de los clientes"].apply(limpiar_valoracion)
df_map.drop(columns=["Valoracion media de los clientes"], inplace=True)
df_map[['numero_valoraciones', 'numero_estrellas']] = df_map['valoraciones'].apply(lambda x: pd.Series(extraer_valoraciones_y_estrellas(x)))
df_map = df_map.drop("valoraciones", axis=1)


/tmp/ipykernel_96893/1594725026.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map["valoraciones"] = df_map["Valoracion media de los clientes"].apply(limpiar_valoracion)
/tmp/ipykernel_96893/1594725026.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map[['numero_valoraciones', 'numero_estrellas']] = df_map['valoraciones'].apply(lambda x: pd.Series(extraer_valoraciones_y_estrellas(x)))
/tmp/ipykernel_96893/1594725026.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling

In [13]:
# Si la 'Capacidad de almacenamiento digital' es igual a 'Memoria RAM', poner None
for index, row in df_map.iterrows():
    if row['Capacidad de almacenamiento digital'] == row['Memoria RAM']:
        df_map.at[index, 'Capacidad de almacenamiento digital'] = None

In [14]:
def corregir_filas(fila):
    # Si la Memoria RAM > 36 y almacenamiento digital < Memoria RAM, intercambia valores
    if fila['Memoria RAM'] > 36:
        valor_a = fila['Memoria RAM']
        fila['Memoria RAM'] = None
        if pd.notna(fila['Capacidad de almacenamiento digital']) and fila['Capacidad de almacenamiento digital'] < valor_a:
            fila['Memoria RAM'], fila['Capacidad de almacenamiento digital'] = fila['Capacidad de almacenamiento digital'], valor_a

    # Si almacenamiento < 36 y < RAM, RAM = almacenamiento y almacenamiento = None
    if pd.notna(fila['Capacidad de almacenamiento digital']) and fila['Capacidad de almacenamiento digital'] < 36:
        if pd.isna(fila['Memoria RAM']) or (pd.notna(fila['Memoria RAM']) and fila['Capacidad de almacenamiento digital'] < fila['Memoria RAM']):
            fila['Memoria RAM'] = fila['Capacidad de almacenamiento digital']
        fila['Capacidad de almacenamiento digital'] = None

    return fila

df_map = df_map.apply(corregir_filas, axis=1)

In [15]:
def extraer_dimensiones_peso(valor):
    try:
        dimensiones, peso = valor.split(';')
        dimensiones = dimensiones.replace('cm', '').strip()
        peso = peso.replace('g', '').replace(',', '.').strip()
        dim_split = [float(d.replace(',', '.').strip()) for d in dimensiones.split('x')]

        if len(dim_split) == 3:
            dim_split.sort()
            return pd.Series([dim_split[2], dim_split[1], dim_split[0], float(peso)])
        else:
            return pd.Series([None, None, None, float(peso)])
    except:
        return pd.Series([None, None, None, None])

df_map[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df_map['Dimensiones del producto'].apply(extraer_dimensiones_peso)
df_map.drop(columns=['Dimensiones del producto'], inplace=True)

/tmp/ipykernel_96893/1386069660.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df_map['Dimensiones del producto'].apply(extraer_dimensiones_peso)
/tmp/ipykernel_96893/1386069660.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map[['Altura (cm)', 'Anchura (cm)', 'Profundidad (cm)', 'Peso (g)']] = df_map['Dimensiones del producto'].apply(extraer_dimensiones_peso)
/tmp/ipykernel_96893/1386069660.py:16: PerformanceWarning: DataFrame is highly fra

In [16]:
def limpiar_resoluciones(df_map, columna):
    patron = re.compile(r'(\d{3,4})\s*[x*]\s*(\d{3,4})')

    def procesar_resolucion(resolucion):
        if pd.isna(resolucion):
            return pd.Series([None, None])
        resolucion = str(resolucion).lower()

        if any(palabra in resolucion for palabra in ["hd+", "pixels", "~", "fhd+", "p"]):
            return pd.Series([None, None])
        
        match = patron.search(resolucion)
        if match:
            return pd.Series([match.group(1), match.group(2)])
        
        # Casos específicos
        if "4k" in resolucion:
            return pd.Series(["3840", "2160"])
        if "1080p" in resolucion:
            return pd.Series(["1920", "1080"])
        if "1k" in resolucion:
            return pd.Series(["1024", "768"])
        return pd.Series([None, None])

    df_map[['ancho', 'alto']] = df_map[columna].apply(procesar_resolucion)

    df_map['ancho'] = pd.to_numeric(df_map['ancho'], errors='coerce')
    df_map['alto'] = pd.to_numeric(df_map['alto'], errors='coerce')

    return df_map

def rellenar_alto_ancho(row):
    if pd.isna(row['alto']) and pd.isna(row['ancho']):
        if not pd.isna(row['Resolucion horizontal']) and not pd.isna(row['Resolucion vertical max']):
            row['alto'] = row['Resolucion vertical max']
            row['ancho'] = row['Resolucion horizontal']
    return row

df_map = limpiar_resoluciones(df_map, "Resolucion")
df_map.drop("Resolucion", axis=1, inplace=True)
df_map = df_map.apply(rellenar_alto_ancho, axis=1)

/tmp/ipykernel_96893/2858698630.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map[['ancho', 'alto']] = df_map[columna].apply(procesar_resolucion)
/tmp/ipykernel_96893/2858698630.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_map[['ancho', 'alto']] = df_map[columna].apply(procesar_resolucion)


## Gestion de nulos

In [17]:
df_null = df_map.dropna(subset=['precio_actual']).copy()
# Tengo una columna mas que en la otra version
df_null.shape

(953, 108)

In [18]:
# Gestion de moda o media para un rango de precios
def rellenar_con_precio(row, df_null, col_nulos, col_comparar, metodo='moda'):
    if pd.isnull(row[col_nulos]):
        precio = row[col_comparar]
        if pd.notnull(precio):
            rango_min = precio * 0.7
            rango_max = precio * 1.3
            df_null_filtrado = df_null[(df_null[col_comparar] >= rango_min) & (df_null[col_comparar] <= rango_max)]
            if metodo == 'moda':
                valor = df_null_filtrado[col_nulos].mode()
                if not valor.empty:
                    return valor[0]
            elif metodo == 'media':
                valor = df_null_filtrado[col_nulos].mean()
                if not pd.isnull(valor):
                    return valor
    return row[col_nulos]

def rellenar_nulos_con_moda(df_null, columna):
    moda = df_null[columna].mode()[0]
    df_null[columna] = df_null[columna].fillna(moda)
    return df_null

def rellenar_nulos_con_media(df_null, columna):
    media = round(df_null[columna].mean(), 2)
    df_null[columna] = df_null[columna].fillna(media)
    return df_null


In [19]:
# Capacidad de almacenamiento digital
col_rellenar = "Capacidad de almacenamiento digital"
col_comparar = "precio_actual"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)

# Capacidad de Potencia nominal de la bateria
col_rellenar = "Potencia nominal de la bateria"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)

# Tamano de la pantalla
col_rellenar = "Tamano de la pantalla"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)
# Tipo de pantalla
col_rellenar = "Tipo de pantalla"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)
# Ano del modelo
df_null = rellenar_nulos_con_moda(df_null, "Ano del modelo")

# Memoria RAM
col_rellenar = "Memoria RAM"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)
# Profundidad, Peso
df_null = df_null.drop(['Altura (cm)', 'Anchura (cm)'], axis=1)
df_null = rellenar_nulos_con_media(df_null, "Profundidad (cm)")
df_null = rellenar_nulos_con_media(df_null, "Peso (g)")

# Precio anterior
df_null['precio_anterior'].fillna(df_null['precio_actual'], inplace=True)

# Valoraciones & Estrellas
col_rellenar = "numero_valoraciones"
df_null[col_rellenar] = df_null.apply(
    lambda row: rellenar_con_precio(row, df_null, col_rellenar, col_comparar), axis=1
)
col_rellenar = "numero_estrellas"
df_null[col_rellenar] = df_null.apply(
    lambda row: round(rellenar_con_precio(row, df_null, col_rellenar, col_comparar, metodo="media"), 1), 
    axis=1
)

df_null['promedio_valoraciones'] = df_null['numero_valoraciones'] / df_null['numero_estrellas']
df_null = df_null.drop(["numero_valoraciones", "numero_estrellas"], axis=1)


# Velocidad de CPU (GHz)
col_rellenar = "Velocidad de CPU (GHz)"
df_null[col_rellenar] = df_null.apply(
    lambda row: round(rellenar_con_precio(row, df_null, col_rellenar, col_comparar, metodo="media"), 1), 
    axis=1
)
# Ancho & Alto px
def unir_columnas(df_null, col1, col2, nueva_col):
    df_null[nueva_col] = df_null[col1].dropna().astype(str) + 'x' + df_null[col2].dropna().astype(str)
    return df_null
df_null = unir_columnas(df_null, 'alto', 'ancho', 'dimensiones')
df_null_dimension = df_null['dimensiones']
moda = df_null_dimension.mode()[0]

moda_altura, moda_anchura = moda.split('x')
moda_altura = float(moda_altura)
moda_anchura = float(moda_anchura)

df_null['alto'] = df_null['alto'].fillna(moda_altura)
df_null['ancho'] = df_null['ancho'].fillna(moda_anchura)
df_null = df_null.drop('dimensiones', axis=1)

/tmp/ipykernel_96893/1018291080.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_null['precio_anterior'].fillna(df_null['precio_actual'], inplace=True)


## Filtramos columnas
Eliminamos las culomnas que tienen un porcentaje de nulos mayor al indicado

In [20]:
threshold = 0.40 * len(df)
df_filtr = df_null.dropna(axis=1, thresh=threshold).copy()
# Valorem de eliminar els nuls que no s'han pogut eliminar per el nosetre criteri
print(f'{df_filtr.dropna().shape[0]}/{df_filtr.shape[0]}')

941/953


In [21]:
df_filtr.dropna(inplace=True)

## Refactoritcació de columnes

In [22]:
df_rename = df_filtr.copy()

In [23]:
df_rename = df_rename.rename(
    columns={
        'Ano del modelo': 'ano', 
        'Capacidad de almacenamiento digital': 'almacenamiento',
        'Tamano de la pantalla':'pantalla_in',
        'Tipo de pantalla':'pantalla_tipo',
        'Potencia nominal de la bateria':'bateria',
        'Memoria RAM':'ram',
        'Profundidad (cm)':'grosor',
        'Peso (g)':'peso',
        'ancho':'ancho_px',
        'alto':'alto_px',
        'Velocidad de CPU (GHz)':'velocidad_cpu_ghz'
        }
    )


df_rename.columns = df_rename.columns.str.lower()
nuevo_orden = [
    'ano', 'almacenamiento', 'marca',
    'pantalla_in', 'pantalla_tipo',
    'velocidad_cpu_ghz', 'ram',
    'grosor', 'peso', 'ancho_px', 'alto_px',
    'bateria',                         
    'promedio_valoraciones',
    'precio_anterior', 'precio_actual'
]
df_rename = df_rename[nuevo_orden]
df_rename

,ano,almacenamiento,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
_id,,,,,,,,,,,,,,,
B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,0.0,4.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,1.2,12.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00


In [24]:
df_rename.to_csv("../res/dataset_cleaned.csv", index = True)

### Insertar MongoDB

In [25]:
df_final = df_rename.copy()

In [ ]:
cliente = MongoClient(MONGO_URI)
db = cliente[DB_NAME]
coleccion = db["products_cleaned"]

df_reset = df_final.reset_index()
data_dict = df_reset.to_dict("records")

def insertar_datos(data_dict, coleccion):
    for data in data_dict:
        coleccion.update_one({'_id': data['_id']},{'$set': data},upsert=True)
    print("Datos insertados o actualizados")

# insertar_datos(data_dict, coleccion)


### Exportar MongoDB

In [ ]:
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
carpeta = "../backup/"
nombre_archivo = f"backup_mobiles_cleaned_{fecha_hora_actual}.json"
destino = carpeta + nombre_archivo

def exportar_datos(documentos, destino):
    for doc in documentos:
        doc['_id'] = str(doc['_id'])
    with open(destino, "w", encoding="utf-8") as archivo:
        json.dump(documentos, archivo, ensure_ascii=False, indent=4)
    print("Base de datos exportada con éxito")

# exportar_datos(list(coleccion.find()), destino)

In [28]:
destino = "../backup/backup_mobiles_cleaned_21_02_2025_19_24.json"
df_mongo = pd.read_json(destino)
df_mongo

,_id,alto_px,ancho_px,ano,bateria,grosor,marca,memoria,pantalla_in,pantalla_tipo,peso,precio_actual,precio_anterior,promedio_valoraciones,ram,velocidad_cpu_ghz
0,B00IRZ8EQC,960,540,2013,5000,1.27,samsung,256,4.30,lcd,77.11,245.00,199.99,279.459459,1,1.7
1,B00J8OA220,320,240,2024,160,10.16,inmarsat,160,2.00,lcd,318.00,933.00,199.99,0.800000,2,2.9
2,B00JC8MD7Y,2400,1080,2024,4500,0.77,samsung,256,6.70,superamoled,179.00,279.99,279.99,1761.818182,6,2.2
3,B00TKALUDC,2400,1080,2015,800,1.80,ttfone,128,2.00,lcd,75.00,41.99,45.99,168.571429,32,0.0
4,B00TUXHZTW,2160,3840,2015,800,1.90,doro,128,2.00,lcd,74.00,48.67,48.67,74.000000,16,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,B0DH3N1GZ3,1080,1920,2024,5000,2.57,honor,128,6.56,lcd,254.42,99.90,199.99,1.702128,4,22.1
937,B09QH71RZR,1080,1920,2024,5000,0.81,xiaomi,128,6.43,amoled,179.00,127.00,165.39,3821.818182,6,2.0
938,B0DK4NBZWB,2400,1080,2024,5000,2.57,oppo,256,6.67,amoled,254.42,299.00,199.99,2.291667,8,3.4
939,B0D8TCGKGN,720,1280,2024,10600,2.57,oukitel,256,6.52,hd,254.42,149.99,199.99,149.772727,24,1.6
